In [1]:
%pip install py2neo

Note: you may need to restart the kernel to use updated packages.


In [2]:
from py2neo import Graph, Node, Relationship

graph = Graph("neo4j+s://b26b71fa.databases.neo4j.io", auth=("neo4j", "YWpXBbSXHIcloDAhR_ZfTo9iR6jLzFIwyux_bNBU6lA"))
query = "MATCH (t:Task {name:\"BugFix\"}) RETURN t"
results = graph.run(query).data()
print(results)

[{'t': Node('Task', description='Fix bug in CatalogService', name='BugFix', status='Open')}]


In [24]:
import torch
from py2neo import Graph
from sentence_transformers import SentenceTransformer, util
from transformers import GPTNeoForCausalLM, AutoTokenizer
import faiss

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')  # Open-source embedding model
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-125M")
gpt_neo = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-125M")

def fetch_task_by_team(team):
    query = f"""
    MATCH (t:Task)-[:ASSIGNED_TO]->(g:Team {{name: '{team}'}})
    RETURN t.name AS TaskName
    """
    results = graph.run(query).data()
    print("query results: " + ', '.join([record['TaskName'] for record in results]))
    return [record['TaskName'] for record in results]

def create_embedding(text):
    return embedding_model.encode(text, convert_to_tensor=True)

def build_faiss_index(documents):
    embeddings = [create_embedding(doc) for doc in documents]
    d = embeddings[0].shape[0]  # Get the embedding dimension
    index = faiss.IndexFlatL2(d)  # Use L2 distance
    index.add(torch.stack(embeddings).cpu().numpy())
    return index, documents

def search_faiss(query, index, documents):
    query_embedding = create_embedding(query).cpu().numpy()
    D, I = index.search(query_embedding.reshape(1, -1), k=3)
    return [documents[i] for i in I[0]]

def generate_response(retrieved_docs, user_query):
    input_text = f"Task Assigned to {user_query}: {', '.join(retrieved_docs)}. What is the status?"
    inputs = tokenizer(input_text, return_tensors="pt")
    output = gpt_neo.generate(**inputs, max_new_tokens=50)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def rag_pipeline(user_query):
    # 1. Retrieve relevant books from Neo4j
    team_tasks = fetch_task_by_team(user_query)
    print("team taska: " + team_tasks[1])
    
    # 2. Build FAISS index on the retrieved documents
    index, documents = build_faiss_index(team_tasks)
    print("documents: " + documents[1])
    print("index: " + index)
    
    # 3. Search FAISS for relevant matches to user query
    retrieved_docs = search_faiss(user_query, index, documents)
    print("retrieved docs: " + retrieved_docs[1])
    
    # 4. Generate response using GPT-Neo
    response = generate_response(retrieved_docs, user_query)
    
    return response

user_input = "TeamC"
response = rag_pipeline(user_input)
print(response)

query results: Refactor, ReviewFix


IndexError: list index out of range